In [3]:
import numpy as np 
import pandas as pd

In [4]:
url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dados_multi_classe.csv"
df = pd.read_csv(url, delimiter=',')

In [5]:
df

,Institution,Affiliations
0,aristotle university greece,1st department of ophthalmology aristotle univ...
1,aristotle university greece,aristotle univ thessaloniki greece
2,aristotle university greece,aristotle univ thessaloniki thessaloniki greece
3,aristotle university greece,aristotle university of thessaloniki greece
4,aristotle university greece,aristotle university of thessaloniki thessalon...
...,...,...
122929,universidade federal reconcavo,"univ fed reconcavo bahia, centro formacao prof..."
122930,unesp,unesp-marilia
122931,universidade federal campina grande,univ federal campina grande
122932,revista saude publica,revista de saude publica


In [6]:
fiocruz = df[df['Institution'].str.contains("fiocruz")]

In [7]:
fiocruz.shape[0]

20745

In [8]:
fiocruz.sample(n=10)

,Institution,Affiliations
21988,fiocruz mg rené rachou cpqrr,centro de pesquisa rené rachou cpqrr fiocruz b...
27475,fiocruz pr instituto carlos chagas icc,laboratory of applied science and technology i...
18865,fiocruz am instituto leônidas e maria deane ...,instituto de pesquisa leônidas e maria deane f...
17655,farmanguinhos instituto de tecnologia em fárm...,instituto de tecnologia em fármacos farmanguin...
14573,ensp escola nacional de saúde pública sérgio ...,fundação oswaldo cruz centro de estudos em saú...
25699,fiocruz pe instituto aggeu magalhães cpqam,department of immunology aggeu magalhães insti...
15298,ensp escola nacional de saúde pública sérgio ...,ppg public health and environment escola nacio...
23192,fiocruz mg rené rachou cpqrr,instituto rené rachou irr fundação oswaldo cru...
17773,farmanguinhos instituto de tecnologia em fárm...,laboratory of development and analytical valid...
62245,procc programa de computação científica fiocruz,fundação oswaldo cruz programa de computação c...


In [9]:
fiocruz.to_csv("dicionario_fiocruz.tsv", sep = "\t", index=False)